# Time Domain Measurements

## Notes

#### The delay (in number of digitizer samples) needed to start acquisition of I and Q signal after marker arrival at the digitizer was determined to be 125 samples = 250ns. Briefly, 20dB of attenuation after readout up-conversion, without using the on-rack 17dB amplifier (before down-conversion), was enough to obtain a 40mVpp pulse with IF amplitude = 1. This signal was then measured with the digitizer (i.e., the discrete voltage levels of the pulse), and the delay was determined to be 125 samples (250ns). The normal rack configuration (needed to reach low photon levels) uses 40dB of attenuation after up-conversion, along with a ~17dB amplifier prior to downconversion. The normal configuration leads to really noisy output (you can't make out a pulse by visual inspection of the signal), but with the demodulation to DC plus averaging, you can really pick out the IF tone

#### We put attenuators on the XY line. Through testing, we saw that, even in the abscence of a pulse, we saw the frequency response of the cavity change in comparison to when sigXY is off AT VARIOUS FREQUENCIES. This implies carrier bleedthrough and the spectrum analyzer confirms that. To ameliorate the issue, we decreased LO power while staying within mixer spec, added attenuation after XY upconversion, and increased the AWG amplitude voltage up to the 1dB compression point of the mixer. This basically maximizes the ratio in power of the desired mixer sideband vs the LO bleedthrough while staying within mixer spec. Increasing the IF also further isolates the qubit from carrier bleedthrough. Typical attenuations range from 10dB-30dB, and depend on the coupling between the qubit and the XY line on chip. 

## Equipment Initialization

In [ ]:
using Revise # for module revision without restarting notebook/Julia
using InstrumentControl
using InstrumentControl:E5071C, GS200, SMB100A, AWGM320XA, DigitizerM3102A 
using AxisArrays
using KeysightInstruments
using QuTroller
using JLD, FileIO, LsqFit
import Plots
Plots.plotlyjs()

import InstrumentControl: Stimulus, Response, Instrument, source, measure, PropertyStimulus

IQ_to_dBm(x) =  10+20*log10(abs(x)) #to be used on digitizer IQ output
phas(x)=atan2(imag(x),real(x))

### Yokogawa GS200 DC Source

In [ ]:
yoko = InsGS200(tcpip_socket("169.254.65.200",7655))  #initializing instrument object
Vz = RampStim(yoko) 

### R&S SMB100A Signal Generators

In [ ]:
#iinitializing signal generator instrument objects
sigRead = InsSMB100A(tcpip_instr("169.254.1.20")) 
sigXY = InsSMB100A(tcpip_instr("169.254.235.224"))

In [ ]:
#defining stimuli for changing signal generator power and frequencies
LOxyPower = PropertyStimulus(sigXY, PowerLevel, axisname=:xypower, axislabel="XY LO Power (dBm)")
LOxyFreq = PropertyStimulus(sigXY, Frequency, axisname=:xyf, axislabel="XY LO Frequency (GHz)");
LOreadFreq = PropertyStimulus(sigRead, Frequency, axisname=:readf, axislabel="Readout LO Frequency (GHz)");
LOreadPower = PropertyStimulus(sigRead, PowerLevel, axisname=:readoutpower, axislabel="Readout LO Power (dBm)")

### Keysight Hardware

In [ ]:
#The inputs to the constructors are slot numbers and chassis number
awg3 = InsAWGM320XA(3, 0) 
awg5 = InsAWGM320XA(5, 0)
awg7 = InsAWGM320XA(7, 0)
dig = InsDigitizerM3102A(18, 0)

### QuTroller Initialization

In [ ]:
Qcon = QuTroller.qubitController[]

Q1 = Qubit(Qcon, awg3, 3, 4, "left qubit")
Q2 = Qubit(Qcon, awg5, 1, 2, "right qubit")

Qcon[RO] = (awg3, 1, 2, sigRead)
Qcon[Marker] = (awg7, 4)
Qcon[Digitizer] = (dig, 1, 2)
Qcon[xyLOsource] = sigXY

In [ ]:
Qcon[ReadoutLength] = 500e-9
Qcon[ReadoutIF] = 100e6
Qcon[ReadoutPower] = 6 #in dBm
Qcon[DecayDelay] = 30e-6
Qcon[EndDelay] = 30e-6

Qcon[Averages] = 2000
Qcon[DigDelay] = 125 #delay between digitizer receving trigger and data acquisition, in number of samples 
Qcon[PXI] = 0

Qcon["left qubit", xyIF] = 100e6
Qcon["left qubit", xyAmplitude] = 0.631 #in Volts
Qcon["right qubit", xyIF] = 100e6
Qcon["right qubit", xyAmplitude] = 0.631 #in Volts

In [ ]:
readRef = ReadoutReference(5e-6) #input is delay between readout pulses
_avgIQ = Avg_IQResponse(RO_IQ_Response(:TRGPort, 100e6))

## Qubit 1 ("left qubit") Characterization

### Finding Resonator Frequency

In [ ]:
source(readRef)
sweep(_avgIQ, (LOreadFreq, (6.980e9:0.1e6:6.989e9)))

In [ ]:
r = result(6257)
Plots.plot(axisvalues(r)[1]/1e9, IQ_to_dBm.(r), xl="Readout LO Frequency (GHz)", yl="IF Power (dBm)")

In [ ]:
Qcon[ReadoutLO] = 6.9867e9 #update LO frequency with result of sweep

##### |0> dispersed resonator frequency:

### Finding Qubit Frequency 

In [ ]:
source(Rabi("left qubit"), 200e-9) 
sweep(_avgIQ, (LOxyFreq, 4.65e9:0.5e6:4.9e9))

In [ ]:
r = result(6292)
Plots.plot(axisvalues(r)[1]/1e9, IQ_to_dBm.(r), xl="XY LO Frequency (GHz)", yl="IF Power (dBm)")

In [ ]:
Qcon[xyLO] =  4.87e9 #update LO frequency with result of sweep

##### Qubit frequency:

### Finding Rabi Period

In [ ]:
sweep(_avgIQ, (Rabi("left qubit"), 20e-9:10e-9:2e-6))

In [ ]:
r = result(6293)
Plots.plot(axisvalues(r)[1]*1e9, abs.(r), xl="Time (ns)", yl="IF Amplitude" )

##### Pi pulse length: 

### Chevron Pattern

In [ ]:
sweep(_avgIQ, (Rabi("left qubit"), 20e-9:10e-9:2e-6), (LOxyFreq, 4.86e9:1e6:4.88e9))

In [ ]:
r = result(6239)
Plots.heatmap(axisvalues(r)[1]*1e9, (axisvalues(r)[2] - Qcon[xyLO])/1e9, IQ_to_dBm.(r), xl="Pulse duration (ns)", yl="XY Frequency (GHz)")

### Measuring T1

In [ ]:
Qcon["left qubit", X] = 200e-9 #set up π pulse length you want for T1
sweep(_avgIQ, (T1("left qubit"), 20e-9:20e-9:20e-6))

In [ ]:
r = result(6308)
Plots.plot(axisvalues(r)[1]*1e6, abs.(r), xl = "Delay (μs)" , yl = "IF Amplitude")

In [ ]:
Plots.plot(axisvalues(r)[1]*1e6, abs.(r), xl = "Delay (μs)", yl = "IF Amplitude", label = "Data")
t1fit(x,p) = @. p[1]*exp(-x/p[2])+p[3]
fit = curve_fit(t1fit, axisvalues(r)[1]*1e6, Float64.(abs.(r)), [0.0029, 20, 1])
Plots.plot!(axisvalues(r)[1]*1e6, t1fit(axisvalues(r)[1]*1e6, fit.param), label = "Fit T1 ="*string(fit.param[2])[1:4]*"μs")

### Measuring T2*

In [ ]:
Qcon["left qubit", X_2] = 100e-9 #set up π/2 pulse length you want for Ramsey
sweep(_avgIQ, (Ramsey("left qubit"), 20e-9:20e-9:10e-6))

In [ ]:
r = result(6308)
Plots.plot(axisvalues(r)[1]*1e6, abs.(r), xl = "Delay (μs)" , yl = "IF Amplitude")

In [ ]:
t2fit(x,p) = @. p[1]*exp(-x/p[2])*cos(x*(2π/p[3])-p[4])+p[5]
fit = curve_fit(t2fit, axisvalues(r)[1]*1e6, Float64.(abs.(r)), [1, 1 ,1, 1, 0.00286358])
Plots.plot(axisvalues(r)[1]*1e6, abs.(r), xl = "Delay (ns)" , yl = "IF Amplitude", label = "Data")
Plots.plot!(axisvalues(r)[1]*1e6, t2fit(axisvalues(r)[1]*1e6, fit.param), label = "Fit T2 ="*string(fit.param[2])[1:4]*"μs")